**Utilities**

In [1]:
from anytree import Node, Resolver, TreeError, RenderTree, AsciiStyle
from enum import Enum, auto
from collections import deque

"""
Node's hidden properties:
    parent, children, siblings, ancestors, descendants
    root, leaves, is_root, is_leaf
    height, depth, path, iter_path_reverse (iterate **up** the tree from here)
I'll also give it a kwarg 'size': int.

PostOrderIter lists leaves first. Good for calculating directory size efficiently.
    Use filter_=lambda n: n.is_leaf
"""

def read_line_s(file):
    line = f.readline()
    if line == '':
        raise EOFError
    return line.rstrip().split(' ')


class Mode(Enum):
    CMD = auto()
    RESP = auto()


class FileStructure:
    """
    Implementation of the entire file structure. 'anytree' left the app responsible for too much.

    Properties: pwd, root
    Methods: add('name', size=1), cd('name')
    """

    def __init__(self, root_name: str=''):
        self._root = Node(root_name, size=-1)
        self._resolver = Resolver('name')
        self._pwd = self._root

    def __repr__(self):
        _repr = deque()
        for pre, _, node in RenderTree(self._root, maxlevel=3):
            _repr.append(f'{pre}{node.name}')
        return '\n'.join(_repr)

    def render(self, maxlevel: any=None):
        """
        Same as __repr__ but this supports 'maxlevel'.
        """
        _repr = deque()
        for pre, _, node in RenderTree(self._root, maxlevel=maxlevel):
            _repr.append(f'{pre}{node.name}')
        return '\n'.join(_repr)

    def add(self, name: str, size: int=-1):
        try:
            Node(name, parent=self._pwd, size=size)
        except TreeError:
            raise ValueError(f'{name} already exists in file tree.')

    def cd(self, name: str):
        _base = self._root
        if name == '/':
           name = ''  # anytree uses '' as the root name, but the caller doesn't
        elif name in ['.', '..']:
            _base = self._pwd  # relative paths start at pwd
        self._pwd = self._resolver.get(_base, name)

    @property
    def pwd(self) -> str:
        return self._pwd.name

    @property
    def root(self) -> str:
        return self._root.name

# Test the class
test = FileStructure()
test.add('one-one', size=1)
test.add('one-two', size=12)
test.cd('one-one')
test.add('two-one', size=21)
test.add('two-two', size=22)
print(test.render())


├── one-one
│   ├── two-one
│   └── two-two
└── one-two


**Part 1:**

In [2]:
# Build the file tree
dir = FileStructure()
mode = Mode.CMD
with open('../inputs/day7-input') as f:
    # State machine (flat design)
    while True:

        try:
            pieces = read_line_s(f)
        except EOFError:
            break

        print(f'/{dir.pwd} → ' + ' '.join(pieces))  # For debugging

        if mode == Mode.RESP:
            # Response mode
            if pieces[0] == 'dir':
                dir.add(pieces[1])
                continue
            elif pieces[0].isdigit():
                dir.add(pieces[1], pieces[0])
                continue
            else:
                mode = Mode.CMD

        # Command mode
        if pieces[0] == '$':
            if pieces[1] == 'cd':
                dir.cd(pieces[2])
            elif pieces[1] == 'ls':
                mode = Mode.RESP  # Switch to Response mode for next line
            else:
                raise ValueError(f'{pieces[1]} is not a valid input command.')

# Sum file sizes
# TODO

/ → $ cd /
/ → $ ls
/ → dir fwbjchs
/ → dir hmnpr
/ → dir jtrbrcjl
/ → dir lcgv
/ → dir ldqc
/ → dir vrvl
/ → $ cd fwbjchs
/fwbjchs → $ ls
/fwbjchs → 154619 wqdlv.mdw
/fwbjchs → 21648 wvbnz
/fwbjchs → $ cd ..
/ → $ cd hmnpr
/hmnpr → $ ls
/hmnpr → 178623 rftqqsrp.bfm
/hmnpr → $ cd ..
/ → $ cd jtrbrcjl
/jtrbrcjl → $ ls
/jtrbrcjl → dir nmbfwc
/jtrbrcjl → dir whqb
/jtrbrcjl → $ cd nmbfwc


ChildResolverError: Node('/', size=-1) has no child nmbfwc. Children are: 'fwbjchs', 'hmnpr', 'jtrbrcjl', 'lcgv', 'ldqc', 'vrvl'.